# Recursive Animations and The Towers of Hanoi

**Authors:** Yixin Liu, Ruihan Xiao, Frankie Fennell, Eleanor Jeffery, Maia Gray

## Contents

- Introduction
    - Preliminaries
- Section 1: The Sierpinski Triangle
- Section 2: Fractal Trees
- Section 3: Julia Sets
- Section 4: The Towers of Hanoi
- Section 5: Animating the Towers of Hanoi Solution
- Extension: Smooth Animation
- Conclusion
    - Bibliography

## Introduction

Visual representations of concepts can be some of the most engaging and beautiful areas of mathematics. Animations of visualisations can provide insight into the solutions of problems which may be difficult and cumbersome to represent numerically. 

In this project, we first explore how to generate and display images of fractal objects. The formation of fractal shapes are defined by recursive processes, so the shapes are self-similar at all levels of depth. Since we can only perform a limited amount of computation in this project, we will not depict any full fractal shapes - but we perform animations to gain insight into how they are formed. We produce graphical representations of the Sierpinski Triangle, a fractal tree, and of the variation of Julia sets. 

We then focus the tools of recursive functions and animation to producing a graphical solution to the Towers of Hanoi problem.

**Note:** This project contains many animations, each of which may take some time to load

### Preliminaries

We start by importing the necessary libraries, including the python graphical library `pygame`, which will be used in all of the sections of this project. This will be necessary to produce the graphical and interactive animations required.

In [1]:
import random, pygame, os
import numpy as np 
import math
import matplotlib.pyplot as plt
from pathlib import Path

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Section 1: The Sierpinski Triangle

A common example of a fractal object, formed from a recursive process, is the Sierpinski triangle. This object consists of a triangle with sides of a set length, inside which a smaller triangle is first drawn, dividing the larger triangle into four identical triangular-shaped regions. Three more triangles are then drawn in each space between the original triangle and the first triangle drawn, dividing those three spaces into four identical regions respectively. This process is then repeated to infinity to form the full Sierpinski triangle; for this project we will only repeat the process to a limited depth, but we will explore how to animate this process by adapting given functions and using the `pygame` library.

**(Core)** In this section, we are given a `pygame` animation function `draw_sierpinski` that draws the Sierpinski triangle. We are asked to develop this function so that the user is also able to change the speed of the animation, and so that colours are added to the triangle drawing.

**(Explanation)** This section contains an extension of the Sierpinski triangle animation given from the notes. The triangle is built recursively by repeatedly subdividing each triangle into three smaller ones, and the function `make_sierpinski` generates a list of all the triangles required for a chosen depth.

Once the full list is constructed, the animation is handled by drawing each small triangle one at a time using `pygame`, so the fractal gradually appears on the screen rather than showing the whole completed triangle at once.

Two features were added beyond the original code: adjustable animation speed through a global variable `SPEED_FACTOR`, and colour for each triangle based on how deep it sits in the recursion. The user can set the desired depth, start or pause the animation with the space bar, and choose a suitable speed. These additions make the recursive structure and drawing process clearer to observe.

We first define a function `make_sierpinski` which uses an iterative process to form the triangle, storing the data as a list of the vertex coordinates of each triangle along with its colour. An additional function `color_at` defines what colour the triangles at each depth should be drawn in.

In [2]:
def color_at(level: int, depth_max: int) -> tuple[int, int, int]:
    t = level / max(1, depth_max)
    r = int(0   * (1 - t) + 200 * t)
    g = int(120 * (1 - t) +   0 * t)
    b = int(255 * (1 - t) + 200 * t)
    return r, g, b

def make_sierpinski(depth, triangle, triangle_list, level=0, depth_max=None):
    if depth_max is None:
        depth_max = depth


    (x0, y0), (x1, y1), (x2, y2) = triangle


    if depth == 1:
        col = color_at(level, depth_max)
        verts = [(int(x0), int(y0)), (int(x1), int(y1)), (int(x2), int(y2))]
        triangle_list.append((verts, col))
        return

    ABm = ((x0 + x1) / 2.0, (y0 + y1) / 2.0)
    BCm = ((x1 + x2) / 2.0, (y1 + y2) / 2.0)
    CAm = ((x2 + x0) / 2.0, (y2 + y0) / 2.0)


    make_sierpinski(depth-1, ((x0, y0), ABm, CAm), triangle_list, level+1, depth_max)
    make_sierpinski(depth-1, (ABm, (x1, y1), BCm), triangle_list, level+1, depth_max)
    make_sierpinski(depth-1, (CAm, BCm, (x2, y2)), triangle_list, level+1, depth_max)

We then define a function `draw_sierpinski` which utilises `pygame` to animate the triangle's formation.

In [3]:
SPEED_FACTOR = 4
def draw_sierpinski(depth=6):
    '''
    Function that draws the Sierpinski triangle as an animation. 
    The depth of the triangle (recursion) can be adjusted by entering 
    a depth integer value (in [1,10]) as a parameter. 
    For example: python sierpinski.py 8 
    '''
    
    dimensions = (900, 862)
    backgroundColour = (255,255,255)
    blue, black = (0,0,255), (0,0,0)
    # This is the overall outline triangle
    master_triangle = ((50,800),(850,800),(450,62))
    min_depth, max_depth = 1, 10
    clock = pygame.time.Clock()
    warning = "Depth must be an integer in the interval [1,10]"

    if depth < min_depth: 
        depth = min_depth
        print(warning)
        print("Using depth {}".format(min_depth))
    if depth > max_depth: 
        depth = max_depth
        print(warning)
        print("Using depth {}".format(max_depth))

    global SPEED_FACTOR
    try:
        speed_factor = int(SPEED_FACTOR)
    except NameError:
        speed_factor = 4
    # Defines the speed of the animation (see the animation loop) 
    frames_per_second = 10 * speed_factor
    # Make a list of all the triangle vertex coordinates of the given 
    # depth (in make_sierpinski we process  depth to work down to 1)
    triangle_list = []
    make_sierpinski(depth,master_triangle,triangle_list)

    # Initialise pygame and the screen display object and title
    pygame.init()
    screen = pygame.display.set_mode(dimensions)
    # Put the title and instructions for the animation in the title bar of the animation.
    caption = 'Sierpinski Triangle            '
    caption += '(1)  \'Space\' to start or pause    '
    caption += '(2)  Further keystroke instruction here?'
    pygame.display.set_caption(caption)

    # Initialise the display 
    screen.fill(backgroundColour)
    pygame.display.flip()

    # Total number of triangles to be drawn 
    number_of_triangles = len(triangle_list)
    index = 0
    draw_triangle = False
    keep_running = True

    # Animation loop 
    while keep_running:
        for event in pygame.event.get():
            # Exit (at end of this iteration) using quit (e.g Ctrl-q or red button)
            if event.type == pygame.QUIT:
                keep_running = False
            # Start and pause the animation with the space key 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                draw_triangle  = not draw_triangle 

        # Keep draw next triangle with index 'index' if not told to pause and not complete
        if draw_triangle and index  < number_of_triangles:
            verts, col = triangle_list[index] 
            pygame.draw.polygon(screen, col,  verts, 0)
            pygame.draw.polygon(screen, (0,0,0), verts, 1)
            pygame.display.update()
            index += 1

             # Pause time before next iteration starts: one clock tick  
            clock.tick(frames_per_second)
    pygame.quit()
    return None

Testing the animation function:

In [4]:
draw_sierpinski(1)

We then define a new function `run_sierpinski` which allows the user to set the desired depth of the triangle before running the animation. If the input depth is not within the allowed range of 1-10, a default depth of 6 is chosen.

In [5]:
def run_sierpinski(): 
    min_depth, max_depth = 1, 10
    default_depth = 6
    # Get the depth from the user 
    try:
        # If either of the following lines fails then the body of the except statement is run
        depth = int(input("Enter a depth (from {} to {}): ".format(min_depth,max_depth)))
        assert min_depth <= depth <= max_depth
    except:
        print("There was a problem with your input.", end = " ") 
        print("Using default depth:{}".format(default_depth))
        depth = default_depth
    # Now run the animation with the depth input by the user
    draw_sierpinski(depth) 
    return None

Testing the function with a user input:

In [6]:
run_sierpinski()


Enter a depth (from 1 to 10):  1


We now develop the existing function so the user is able to input their desired speed of the animation. Again, if this input is not within the allowed range of 1-10, a default speed of 5 is chosen.

Adding a choice where the user could change the speed of the animation:

In [7]:
def speed_sierpinski():
    min_speed, max_speed = 1, 10
    default_speed = 5
    global SPEED_FACTOR       

    try:
        s = int(input(f"Enter a speed (from {min_speed} to {max_speed}): "))
        assert min_speed <= s <= max_speed
        SPEED_FACTOR = s        
    except Exception:
        print("There was a problem with your input. Using default:", default_speed)
        SPEED_FACTOR = default_speed

    run_sierpinski()
    return None

In [ ]:
speed_sierpinski()

Meanwhile, we have added colours to triangle drawing. We have done this by adding code to the function `make_sierpinski` so that colour is added during the construction process, instead of in the other stages. Note that the function `make_sierpinski` here is now different from that in the document given: it has been amended to add colour as well.

## Section 2: Fractal Trees

Fractal shapes are often used to represent biological objects, as they can provide the level of detail similar to that found in nature better than Euclidean shapes can. Trees provide examples of natural objects with fractal geometry, as described by Mandelbrot in $[1]$. We construct an image of a simple fractal tree through a similar recursive process as in the previous section.

 **(Core)** In this section, we develop a `pygame` animation that constructs a simple recursively defined tree. The tree defined to depth 1 is just a trunk with three straight branches. Then, given a tree $T$ of depth $n$, the tree of depth $n + 1$ is the tree $T$ with every branch replaced by a tree of depth 1. The tree is coloured so that its 'branches' are brown in the animation, and its 'leaves' (the last layer of branches) green. We add user inputs, so that the user is able to choose the depth of the tree, and has control over the speed of the animation, as well as being able to start and stop it.

 **(Explanation)** In this section we build the recursive tree required in the question. The idea is quite straightforward: each branch is represented as a line from one point to another, and for the next depth we let every branch grow three new branches at its tip. The new branches are shorter than the previous one and they point in different directions given by fixed angles. Repeating the process in this way creates the fractal-like structure.

The function `make_tree` handles this construction. Starting from the trunk, it works out the length and direction of the current branch. If we reach the final depth, it adds three short “leaf” branches and marks them as leaves. Otherwise it creates three child branches, marks them as ordinary branches, and then calls itself recursively on each of them. By the end, this gives a list of line segments together with a tag telling us whether the segment is a brown branch or a green leaf.

For the animation part we follow the same style as in Section 1. The function `animate_tree` opens a `pygame` window, and then draws the segments one by one, so the user can actually see the tree being constructed. Branches are drawn in brown and the leaves in green, as required. The user can press the space bar to start or pause the animation, and `Esc` to quit. Simple speed control has been added through a global `SPEED_FACTOR`, which directly affects the frame rate so that the tree can be drawn faster or more slowly, depending on the user’s choice.

The helper functions `run_tree()` and `speed_tree()` allow the user to enter the depth and the speed before starting the animation, keeping the interface consistent with the Sierpinski triangle functions in the previous section.

In [8]:
def make_tree(depth, branch, tree_list,
              angles=(-35.0, 0.0, 35.0), scale=0.72,
              include_internal=True):
    (x0, y0), (x1, y1) = branch
    dx, dy = x1 - x0, y1 - y0
    L = math.hypot(dx, dy)
    heading = math.degrees(math.atan2(dy, dx))
    tip = (x1, y1)

    if depth <= 1:
        leaf_len = L * 0.4
        for a in angles:
            rad = math.radians(heading + a)
            x2 = tip[0] + leaf_len * math.cos(rad)
            y2 = tip[1] + leaf_len * math.sin(rad)
            tree_list.append(((tip, (x2, y2)), 'leaf'))
        return tree_list

    child_len = L * scale
    for a in angles:
        rad = math.radians(heading + a)
        x2 = tip[0] + child_len * math.cos(rad)
        y2 = tip[1] + child_len * math.sin(rad)
        child = (tip, (x2, y2))
        if include_internal:
            tree_list.append((child, 'branch'))
        make_tree(depth-1, child, tree_list,
                  angles=angles, scale=scale,
                  include_internal=include_internal)
    return tree_list


We next define the animation function:

In [9]:
SPEED_FACTOR = 4
BROWN = (139, 69, 19) 
GREEN = (34, 139, 34) 

def animate_tree(depth=6):
    root_branch = ((450.0, 820.0), (450.0, 620.0))
    segs = []
    make_tree(depth, root_branch, segs, angles=(-35,0,35), scale=0.72, include_internal=True)

    pygame.init()
    screen = pygame.display.set_mode((900, 862))
    pygame.display.set_caption("Recursive Tree (Space: start/pause, Esc: quit)")
    screen.fill((255,255,255)); pygame.display.flip()
    clock = pygame.time.Clock()

    fps = 10 * int(SPEED_FACTOR)
    i, n = 0, len(segs)
    drawing, running = False, True
    while running:
        for e in pygame.event.get():
            if e.type == pygame.QUIT: running = False
            elif e.type == pygame.KEYDOWN:
                if e.key == pygame.K_SPACE: drawing = not drawing
                elif e.key == pygame.K_ESCAPE: running = False

        if drawing and i < n:
            (p0, p1), tag = segs[i]
            color = GREEN if tag == 'leaf' else BROWN
            pygame.draw.line(screen, color, (int(p0[0]),int(p0[1])), (int(p1[0]),int(p1[1])), 1)
            pygame.display.update()
            i += 1
            clock.tick(fps)

    pygame.quit()


Testing the animation function to a depth of 7:

In [10]:
animate_tree(depth=7)

The next function allows the user to input their desired depth, as in Section 1.

In [11]:
def run_tree(): 
    min_depth, max_depth = 1, 10
    default_depth = 6
    # Get the depth from the user 
    try:
        # If either of the following lines failsthen the body of the except statement is run
        depth = int(input("Enter a depth (from {} to {}): ".format(min_depth,max_depth)))
        assert min_depth <= depth <= max_depth
    except:
        print("There was a problem with your input.", end = " ") 
        print("Using default depth:{}".format(default_depth))
        depth = default_depth
    # Now run the animation with the depth input by the user
    animate_tree(depth) 
    return None

Testing the animation, allowing the user to input their choice of the depth:

In [12]:
run_tree()

Enter a depth (from 1 to 10):  1


Again we define a new function to allow the user to also input their desired speed.

In [13]:
def speed_tree():
    min_speed, max_speed = 1, 10
    default_speed = 5
    global SPEED_FACTOR       

    try:
        s = int(input(f"Enter a speed (from {min_speed} to {max_speed}): "))
        assert min_speed <= s <= max_speed
        SPEED_FACTOR = s        
    except Exception:
        print("There was a problem with your input. Using default:", default_speed)
        SPEED_FACTOR = default_speed

    run_tree()
    return None

Testing the animation, allowing the user to input the speed of the animation as well as the depth:

In [14]:
speed_tree()

Enter a speed (from 1 to 10):  2
Enter a depth (from 1 to 10):  2


## Section 3: Julia Sets

Julia sets are examples of fractal objects produced through numerical means. We now extend our animation methods to create not just a single image as before, but a sequence of images to produce a film.

**(Core)** In this section we investigate how to produce and plot Julia sets, and display a sequence of images of these sets as an animated film. These Julia sets have parameter $j_p = 0.7885e^{{\alpha}i}$, for small non-negative $\alpha \in \mathbb{R}$. We generate 200 image files - one for each Julia set as $\alpha$ takes 200 equally spaced values between $0$ and $2\pi$. These are then played in a sequence, using a `pygame` animation, to produce a film of the sets as they vary. 

**(Explanation)** In this section we create code that generates animated Julia set fractals as required in the question.
The three main parts are: `julia_escape_smooth`, which creates a smooth Julia fractal image; `generate_julia_frames`, which creates and saves a sequence of Julia set images; and `play_frames_pygame`, which plays the saved images.

The `julia_escape_smooth` function creates a grid of complex numbers (`Z`) and iterates the function, tracking how quickly each point escpaes beyond the radius **R**, using a smooth gradient. The points that don't escape before reaching the maximum iteration `max_iter` are assigned their maximum value.  The function returns a 2D array of escape-time values that can be plotted as a fractal image. 

The function `generate_julia_frames` creates an output folder, and generates a number of evenly spaced angles $\alpha$ from $0$ to $ 2  π $, which is defined as `n_frames` - this value is initialised to 200 when the function is run. For each angle, it computes $c = r e ^{(ia)}$ and calls `julia_escape_smooth` to create a fractal image for this value of $\alpha$. It then saves each frame as `julia_000.png`, `julia_001.png`, etc. and returns a folder storing the images. These images can then be displayed in sequence to produce an animation of the variation of the sets. 

The `play_frames_pygame` function uses `pygame` to load all the images in the folder and play back the frames like a video, repeating the process in a loop. It also allows for interactive control:

- **Quit/Esc** - closes the window
- **Space** - pauses/plays the animation
- **Up/Down arrow** - increases/decreases the speed of the animation


In [7]:
def julia_escape_smooth(c, max_iter=200, R=2.0,
                        xlim=(-1.6, 1.6), ylim=(-1.2, 1.2), res=900):
    x = np.linspace(*xlim, res)
    y = np.linspace(*ylim, res)
    X, Y = np.meshgrid(x, y)
    Z = X + 1j*Y

    count = np.zeros(Z.shape, dtype=float)
    alive = np.ones(Z.shape, dtype=bool)

    for n in range(max_iter):
        Z[alive] = Z[alive]**2 + c
        mag = np.abs(Z)
        esc = mag > R
        just = esc & alive
        if np.any(just):
            count[just] = n + 1 - np.log2(np.log(mag[just]) / np.log(R))
        alive &= ~esc
        if not alive.any():
            break

    count[alive] = max_iter
    return count

def generate_julia_frames(n_frames=200, r=0.7885, outdir="julia_frames",
                          max_iter=200, res=900, cmap="plasma"):
    out = Path(outdir); out.mkdir(exist_ok=True)
    angles = np.linspace(0, 2*np.pi, n_frames, endpoint=False)
    for k, a in enumerate(angles):
        c = r * np.exp(1j*a)
        img = julia_escape_smooth(c, max_iter=max_iter, res=res)
        plt.figure(figsize=(4,4))
        plt.imshow(img, cmap=cmap, origin="lower",
                   extent=(-1.6, 1.6, -1.2, 1.2))
        plt.axis("off"); plt.tight_layout()
        plt.savefig(out / f"julia_{k:03d}.png", dpi=150, bbox_inches="tight")
        plt.close()
    return out

def play_frames_pygame(folder="julia_frames", fps=12):
    files = sorted(Path(folder).glob("julia_*.png"))
    if not files:
        print("No frames found in", folder)
        return

    pygame.init()
    try:
        imgs = [pygame.image.load(str(p)) for p in files]
        w, h = imgs[0].get_width(), imgs[0].get_height()
        screen = pygame.display.set_mode((w, h))
        pygame.display.set_caption("Julia sets (Space: play/pause, ↑/↓: speed, Esc: quit)")
        clock = pygame.time.Clock()

        i, N = 0, len(imgs)
        playing = False
        running = True
        while running:
            for e in pygame.event.get():
                if e.type == pygame.QUIT:
                    running = False
                elif e.type == pygame.KEYDOWN:
                    if e.key == pygame.K_ESCAPE:
                        running = False
                    elif e.key == pygame.K_SPACE:
                        playing = not playing
                    elif e.key == pygame.K_UP:
                        fps = min(60, fps + 2)
                    elif e.key == pygame.K_DOWN:
                        fps = max(1, fps - 2)

            screen.blit(imgs[i], (0, 0))
            pygame.display.flip()
            if playing:
                i = (i + 1) % N
            clock.tick(fps)
    finally:
        try:
            pygame.display.quit()
        finally:
            pygame.quit()


Testing the function and animation with an initial speed of 12 fps:

In [8]:
generate_julia_frames(n_frames=40, r=0.7885, outdir="julia_frames", max_iter=200, res=500)
play_frames_pygame("julia_frames", fps=12)


## Section 4: The Towers of Hanoi

In the next three sections, we move away from fractals, and apply the methods of recursive functions and animation to solving a problem, rather than depicting an object. The Towers of Hanoi problem involves a stack of discs and three poles. The disks are initially layered on one pole in size order, with the largest at the bottom. The aim is to move the entire stack from one pole to another, following two rules: only one disc can be moved at a time, and a larger disc can never be placed on top of a smaller one.

**(Core)** In this section, we must move $n$ discs from one pole to another, following the rules above.
The function is recursive: move the top $n-1$ discs onto an auxillary pole, move the largest disc to the target, and move the $n-1$ discs onto the largest disc.
The functions generate the move list and give step-by-step pole configurations.

**(Explanation)** We define four functions which produce and display the solution to this problem, showing the configuration of the towers at each individual step. 

The `get_move_direction` function determines whether a disc move is considered left or right; for example, moving from pole 0 to pole 1 would be considered as 'Right', and moving from pole 1 to pole 0 would be 'Left'. 

The `hanoi_solver_matrix` function is a recursive function which generates the list of moves required to reach the solution. `n` indicates the number of discs to move, `source` indicates the index of the starting pole, `target` indicates the index of the final pole, `auxillary` indicates the index of the temporary pole, and `moves_list` stores the list of all moves. 
The recursive step involves moving the $n-1$ discs from the source pole to the auxillary, the $n^{th}$ disc from the source to the target and the remaining $n-1$ discs from the auxillary to the target. 
Each individual move is appended to the list `moves_list`. 

The `print_matrix_state` function converts the current configuration of the poles into an $ n \times 3$ matrix, so that it can be displayed.
It starts by populating an $n \times 3$ matrix with zeros, representing the empty space, and maps the current stack of discs on each pole into the matrix, with the bottom of the stack being the bottom row of the matrix.
The function `solve_with_matrix_view` produces the solution step by step by first initialising the poles, generating each move using `hanoi_solver_matrix`, printing the current state, and repeating the process until the solution is reached.
A final line `time_sleep(0.5)` can be optionally used to slow down the animation.


In [5]:
import time

def get_move_direction(start, end):
    """
    Determine whether the move is 'Left' or 'Right', based on the 
    Wrap Around rule from the lecture notes.
    """
    if (end - start) % 3 == 1:
        return "Right"
    else:
        return "Left"

def hanoi_solver_matrix(n, source, target, auxiliary, moves_list):
    """
    Recursively generate move instructions.
    """
    # Base case: move the single disc directly
    if n == 1:
        direction = get_move_direction(source, target)
        moves_list.append((1, direction, source, target))
        return

    # Step 1: Move n-1 discs from Source to Auxiliary
    hanoi_solver_matrix(n - 1, source, auxiliary, target, moves_list)
    
    # Step 2: Move the nth disc from Source to Target
    direction = get_move_direction(source, target)
    moves_list.append((n, direction, source, target))
    
    # Step 3: Move n-1 discs from Auxiliary to Target
    hanoi_solver_matrix(n - 1, auxiliary, target, source, moves_list)

def print_matrix_state(poles, n, step):
    """
    Core Function: Convert the list of poles (stacks) into an n x 3 matrix and print it.
    This visualizes the towers vertically.
    """
    # 1. Initialize an n x 3 matrix filled with zeros
    # 0 represents empty space (air)
    matrix = [[0] * 3 for _ in range(n)]

    # 2. Populate the matrix
    # We need to map the data from 'poles' into the matrix structure.
    # poles[col] is a stack, e.g., [3, 2, 1] (bottom -> top)
    # In the matrix, the bottom element should be at row index n-1.
    for col_idx in range(3):
        stack = poles[col_idx]
        
        # Iterate through the discs in the current stack
        for i, disc in enumerate(stack):
            # Calculate the row position of the disc in the matrix
            # Logic: If n=3 and current stack has 3 discs:
            # The first disc (stack bottom, i=0) goes to Row 2 (index n-1)
            # The last disc (stack top) goes to a higher row index (smaller number)
            # Formula: row = (n - 1) - i
            row_idx = (n - 1) - i
            matrix[row_idx][col_idx] = disc

    # 3. Print the matrix
    print(f"\n--- Step {step} ---")
    # Display vertically (Top to Bottom)
    for row in matrix:
        # Formatting: Align with spaces.
        # Show 0 as '.' to look like air, non-zero numbers as discs.
        row_str = "  ".join([str(x) if x != 0 else "." for x in row])
        print(f"| {row_str} |")
    print("  0  1  2  (Poles)")

def solve_with_matrix_view(n):
    # 1. Initialize state
    # list(range(n, 0, -1)) generates [n, n-1, ..., 1]
    poles = [list(range(n, 0, -1)), [], []]
    
    # 2. Generate all move instructions
    moves = []
    # Assume moving from Pole 0 to Pole 1 (You can modify the target to 2 if needed)
    hanoi_solver_matrix(n, 0, 1, 2, moves)
    
    # 3. Print initial state
    print(f"Start: {n} discs from Pole 0 -> Pole 1")
    print_matrix_state(poles, n, 0)
    
    # 4. Execute step-by-step and display the matrix
    for step, (disc, direction, start, end) in enumerate(moves, 1):
        # Execute the move in the data structure
        disk_to_move = poles[start].pop()
        poles[end].append(disk_to_move)
        
        # Print the matrix view
        print(f"\nMove: Disc {disc} {direction} ({start}->{end})")
        print_matrix_state(poles, n, step)
        
        # time.sleep(0.5) # Uncomment this line to play the solution slowly, like an animation


In [6]:
# Example Run: n=3
solve_with_matrix_view(3)

Start: 3 discs from Pole 0 -> Pole 1

--- Step 0 ---
| 1  .  . |
| 2  .  . |
| 3  .  . |
  0  1  2  (Poles)

Move: Disc 1 Right (0->1)

--- Step 1 ---
| .  .  . |
| 2  .  . |
| 3  1  . |
  0  1  2  (Poles)

Move: Disc 2 Left (0->2)

--- Step 2 ---
| .  .  . |
| .  .  . |
| 3  1  2 |
  0  1  2  (Poles)

Move: Disc 1 Right (1->2)

--- Step 3 ---
| .  .  . |
| .  .  1 |
| 3  .  2 |
  0  1  2  (Poles)

Move: Disc 3 Right (0->1)

--- Step 4 ---
| .  .  . |
| .  .  1 |
| .  3  2 |
  0  1  2  (Poles)

Move: Disc 1 Right (2->0)

--- Step 5 ---
| .  .  . |
| .  .  . |
| 1  3  2 |
  0  1  2  (Poles)

Move: Disc 2 Left (2->1)

--- Step 6 ---
| .  .  . |
| .  2  . |
| 1  3  . |
  0  1  2  (Poles)

Move: Disc 1 Right (0->1)

--- Step 7 ---
| .  1  . |
| .  2  . |
| .  3  . |
  0  1  2  (Poles)


Note that in this demonstration, the stack was moved from the first pole to the middle pole, instead of to the rightmost pole. However, the process of moving the disks is exactly the same regardless of which pole they are being moved to.

## Section 5: Animating The Towers of Hanoi Solution

Having obtained an algorithm to produce a solution, we now develop an animation to represent the process of this solution.

**(Core)** In this part we are asked to use the list `moves_list` from the previous section to create a pygame animation to display the solution.
We are also asked to allow the user to set the number of discs (betwwen 1 and 16), start and stop the animation, and adjust the speed.

**(Explanation)** The `get_move_direction` function is reused from Section 4.
The function `hanoi_solver_logic` uses a very similar algorithm to the `hanoi_solver_matrix` function from Section 4: move `n-1` discs to the auxillary pole; move the largest disc to the target pole; move the `n-1` discs onto the largest disc.
Here, individual moves are stored as tuples: `(disc_number, direction_string, source_index, target_index)`.
The `get_all-moves` function returns a complete list of moves.

The `pygame` visulisation setup defines the screen size, pole position, and disc size and colours; the `get_disc_colour` function produces a gradient of colours based on the number of discs. This is used to colour the disks according to size order in the visualisation.
The `run_hanoi_animation` function initalises `pygame`, and creates the screen fonts and clock.
User controls are set up in the following way: 

- **Space** - pauses/plays animation 
- **R** - resets animation 
- **Left/Right arrow** - increases/decreases the number of discs 
- **Up/Down arrow** - increases/decreases the speed of animation 
- **Quit** - closes window

The animation is performed one step at a time, with the speed of animation based on the variable `speed_delay`; the discs move directly from one pole to another, without showing intermediate steps, and the animation stops when all the moves have been completed and the solution reached.


In [3]:
import pygame
import time

# --- 1. Core Logic from Part 4 (Reuse) ---
def get_move_direction(start, end):
    """Determines move direction based on wrap-around rule."""
    if (end - start) % 3 == 1:
        return "Right"
    else:
        return "Left"

def hanoi_solver_logic(n, source, target, auxiliary, moves_list):
    """Recursively generates the list of moves."""
    if n == 1:
        direction = get_move_direction(source, target)
        # Storing tuple: (disc_number, direction_str, source_idx, target_idx)
        moves_list.append((1, direction, source, target))
        return

    hanoi_solver_logic(n - 1, source, auxiliary, target, moves_list)
    
    direction = get_move_direction(source, target)
    moves_list.append((n, direction, source, target))
    
    hanoi_solver_logic(n - 1, auxiliary, target, source, moves_list)

def get_all_moves(n):
    """Wrapper to get the full list of moves for N discs."""
    moves = []
    # Moving from Pole 0 to Pole 2 (Standard configuration)
    # Poles: 0 (Left), 1 (Middle), 2 (Right)
    hanoi_solver_logic(n, 0, 2, 1, moves)
    return moves

# --- 2. Pygame Visualization Logic ---

# Constants
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600
BASE_Y = 500
POLE_X_POS = [250, 500, 750]
POLE_WIDTH = 10
POLE_HEIGHT = 300
DISC_HEIGHT = 20
MAX_DISC_WIDTH = 220
MIN_DISC_WIDTH = 40

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (100, 100, 100)
RED = (200, 50, 50)
POLE_COLOR = (80, 50, 20)

def get_disc_color(n, total_discs):
    """Generates a color gradient based on disc size."""
    # Hue shifts from blue/purple to red/orange
    hue = (n / total_discs) * 360
    color = pygame.Color(0)
    color.hsla = (hue, 80, 50, 100)
    return color

def run_hanoi_animation():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Towers of Hanoi - Animation [Bristol Math]")
    clock = pygame.time.Clock()
    font = pygame.font.SysFont("Arial", 24)
    title_font = pygame.font.SysFont("Arial", 32, bold=True)

    # --- State Variables ---
    n_discs = 5          # Initial number of discs
    speed_delay = 0.5    # Seconds between moves (Lower is faster)
    is_playing = False   # Start in paused state
    
    # Initialize Game Data
    poles = [[], [], []]
    moves = []
    move_index = 0
    last_move_time = 0
    
    def reset_game():
        """Resets the poles and recalculates moves based on current n_discs."""
        nonlocal poles, moves, move_index
        # Initialize Pole 0 with discs n, n-1, ..., 1
        poles = [list(range(n_discs, 0, -1)), [], []]
        moves = get_all_moves(n_discs)
        move_index = 0
        
    # Initial reset
    reset_game()

    running = True
    while running:
        current_time = time.time()
        
        # --- 1. Event Handling ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    is_playing = not is_playing
                
                elif event.key == pygame.K_r:
                    reset_game()
                    is_playing = False
                
                elif event.key == pygame.K_RIGHT:
                    if n_discs < 16:
                        n_discs += 1
                        reset_game()
                        is_playing = False
                elif event.key == pygame.K_LEFT:
                    if n_discs > 1:
                        n_discs -= 1
                        reset_game()
                        is_playing = False
                        
                elif event.key == pygame.K_UP:
                    speed_delay = max(0.01, speed_delay - 0.1) # Faster
                elif event.key == pygame.K_DOWN:
                    speed_delay += 0.1 # Slower

        # --- 2. Update Logic (Animation Step) ---
        if is_playing and move_index < len(moves):
            if current_time - last_move_time > speed_delay:
                # Execute ONE move directly (as requested: no intermediate motion)
                # Get move info: (disc_val, direction, src, dst)
                _, _, src, dst = moves[move_index]
                
                # Perform move in data structure
                disc_to_move = poles[src].pop()
                poles[dst].append(disc_to_move)
                
                move_index += 1
                last_move_time = current_time
        
        # If finished
        if move_index >= len(moves) and is_playing:
            is_playing = False # Stop automatically at end

        # --- 3. Drawing ---
        screen.fill(WHITE)
        
        # Draw UI Text
        status_text = "PLAYING" if is_playing else "PAUSED"
        if move_index >= len(moves): status_text = "FINISHED"
        
        info_lines = [
            f"Discs (Left/Right): {n_discs}",
            f"Speed Delay (Up/Down): {speed_delay:.2f}s",
            f"Moves: {move_index} / {len(moves)}",
            f"Status (Space): {status_text}",
            "Press 'R' to Reset"
        ]
        
        for i, line in enumerate(info_lines):
            text_surf = font.render(line, True, BLACK)
            screen.blit(text_surf, (20, 20 + i * 30))

        # Draw Poles
        for x in POLE_X_POS:
            pygame.draw.line(screen, GREY, (x, BASE_Y), (x, BASE_Y - POLE_HEIGHT), POLE_WIDTH)
            # Draw base line
            pygame.draw.line(screen, GREY, (x - 120, BASE_Y), (x + 120, BASE_Y), 5)
            
        # Draw Discs
        # We iterate through the 3 poles (lists)
        for i, stack in enumerate(poles):
            center_x = POLE_X_POS[i]
            
            for j, disc_val in enumerate(stack):
                # Calculate geometry
                # Width depends on disc value (1 is small, n is large)
                # We map 1..n to MIN_WIDTH..MAX_WIDTH
                width_range = MAX_DISC_WIDTH - MIN_DISC_WIDTH
                if n_discs > 1:
                    width = MIN_DISC_WIDTH + (disc_val - 1) * (width_range / (n_discs - 1))
                else:
                    width = MAX_DISC_WIDTH
                
                height = DISC_HEIGHT
                
                # Position: Stack from bottom (BASE_Y) up
                # j is index in stack (0 is bottom)
                rect_x = center_x - (width / 2)
                rect_y = BASE_Y - (j + 1) * height
                
                # Draw
                color = get_disc_color(disc_val, n_discs)
                pygame.draw.rect(screen, color, (rect_x, rect_y, width, height), border_radius=5)
                pygame.draw.rect(screen, BLACK, (rect_x, rect_y, width, height), 1, border_radius=5) # Border

        pygame.display.flip()
        clock.tick(60) # Cap at 60 FPS

    pygame.quit()

if __name__ == "__main__":
    run_hanoi_animation()

## Extension: Smooth Animation

To develop our animation, we can improve the visual representation of the solution by animating the process of each move, instead of purely showing each result. This allows the viewer better understanding of the process, which is one of the main intentions of animating graphical solutions. We also improve the viewer experience by giving them increased control over how to view the animation.

**(Extension 7)** In this extension we are asked to develop the code from Section 5, so that the animation now includes the intermediate steps of the discs moving smoothly between poles.
We also add a menu so the user can choose whether to display the original animation, or the new one including the smooth movements.

**(Explanation)** In this code we introduce a menu which allows the user to set the disc count as before, but also to choose between `playing_instant` (Option 1 - original animation) and `playing_smooth` (Option 2 - new animation).
If the user selects **Option 1**, the logic behind the code is very similar to that in Section 5.

If the user selects **Option 2**, the logic behind the code has three phases to create the smooth movement.
Phase 0 (when `anim_phase == 0`) raises the disc vertically up the pole, phase 1 (when `anim_phase == 1`) moves the disc horizontally to be in line with the new pole, and phase 2 (`when anim_phase == 2`) moves the disc vertically down the pole and starts the next movement.
The user control **R** now returns the user to the menu where they can choose which animation mode to use again.


In [2]:
# =================================================================
# REUSED CORE LOGIC & CONSTANTS (from Part 4 & 5)
# These functions are based on the recursive solution and wrap-around rule.
# =================================================================

# --- Constants ---
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600
BASE_Y = 500
POLE_X_POS = [250, 500, 750]
DISC_HEIGHT = 20
MAX_DISC_WIDTH = 220
MIN_DISC_WIDTH = 40
CLEARANCE_Y = 150 
WHITE = (240, 240, 245)
BLACK = (0, 0, 0)

# --- Core Solver Functions ---
def get_move_direction(start, end):
    """Determines move direction based on wrap-around rule."""
    if (end - start) % 3 == 1: return "Right"
    else: return "Left"

def hanoi_solver_logic(n, source, target, auxiliary, moves_list):
    """Recursively generates the list of moves."""
    if n == 1:
        moves_list.append((1, get_move_direction(source, target), source, target))
        return
    hanoi_solver_logic(n - 1, source, auxiliary, target, moves_list)
    moves_list.append((n, get_move_direction(source, target), source, target))
    hanoi_solver_logic(n - 1, auxiliary, target, source, moves_list)

def get_all_moves(n):
    """Wrapper to get the full list of moves for N discs."""
    moves = []
    hanoi_solver_logic(n, 0, 2, 1, moves) # Default: Pole 0 -> Pole 2
    return moves

def get_disc_color(n, total_discs):
    """Generates a color gradient for visualization."""
    hue = (n / total_discs) * 360
    color = pygame.Color(0)
    color.hsla = (hue, 80, 50, 100)
    return color

# =================================================================
# EXTENSION LOGIC (Part 7: Menu & Smooth Motion)
# This section implements the new features and leverages the functions above.
# =================================================================

def run_extended_hanoi():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Towers of Hanoi: Extension [Bristol Math]")
    clock = pygame.time.Clock()
    
    # Setup Fonts
    font_small = pygame.font.SysFont("Arial", 20)
    font_med = pygame.font.SysFont("Arial", 28)
    font_large = pygame.font.SysFont("Arial", 40, bold=True)

    # --- Game State Variables ---
    # app_state: 'MENU', 'PLAYING_INSTANT', 'PLAYING_SMOOTH'
    app_state = 'MENU' 
    n_discs = 4
    
    # Animation/Movement Variables (Part 5 & 7)
    poles = [[], [], []]
    moves = [] # Uses get_all_moves()
    move_index = 0
    is_paused = False
    speed_delay = 0.5    # Instant Mode: Wait time (Seconds)
    move_speed = 10      # Smooth Mode: Pixel displacement per frame
    last_move_time = 0
    
    # Smooth Motion Tracking (Part 7)
    animating = False
    moving_disc_data = None 
    anim_phase = 0 # 0: Up, 1: Across, 2: Down

    def reset_game():
        """REUSED: Resets poles and recalculates moves using get_all_moves()."""
        nonlocal poles, moves, move_index, animating, moving_disc_data
        poles = [list(range(n_discs, 0, -1)), [], []]
        moves = get_all_moves(n_discs) # FUNCTION REUSED HERE
        move_index = 0
        animating = False
        moving_disc_data = None

    def draw_menu():
        """NEW: Draws the selection menu."""
        screen.fill(WHITE)
        title = font_large.render("Towers of Hanoi Simulation", True, BLACK)
        sub = font_med.render(f"Current Discs: {n_discs} (Use Left/Right to change)", True, BLACK)
        
        opt1 = font_med.render("1. Instant Animation (Part 5)", True, (70, 130, 180))
        opt2 = font_med.render("2. Smooth Animation (Part 7 Extension)", True, (180, 70, 70))
        
        screen.blit(title, (SCREEN_WIDTH//2 - title.get_width()//2, 150))
        screen.blit(sub, (SCREEN_WIDTH//2 - sub.get_width()//2, 220))
        screen.blit(opt1, (SCREEN_WIDTH//2 - opt1.get_width()//2, 300))
        screen.blit(opt2, (SCREEN_WIDTH//2 - opt2.get_width()//2, 350))

    def get_disc_width(val):
        """Helper to calculate disc width based on size and N."""
        if n_discs > 1:
            width_range = MAX_DISC_WIDTH - MIN_DISC_WIDTH
            return MIN_DISC_WIDTH + (val - 1) * (width_range / (n_discs - 1))
        return MAX_DISC_WIDTH
    
    # Initial setup
    reset_game()

    running = True
    while running:
        current_time = time.time()
        
        # --- Event Handling & Menu Navigation ---
        for event in pygame.event.get():
            if event.type == pygame.QUIT: running = False
            
            if app_state == 'MENU':
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_LEFT and n_discs > 1: n_discs -= 1
                    elif event.key == pygame.K_RIGHT and n_discs < 12: n_discs += 1
                    elif event.key == pygame.K_1:
                        reset_game()
                        app_state = 'PLAYING_INSTANT'
                    elif event.key == pygame.K_2:
                        reset_game()
                        app_state = 'PLAYING_SMOOTH'

            elif 'PLAYING' in app_state:
                # Key bindings for pause/speed/reset are handled here.
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE: is_paused = not is_paused
                    elif event.key == pygame.K_r: app_state = 'MENU' # Go back to menu

        # --- Update Logic ---
        if app_state == 'PLAYING_INSTANT':
            # REUSED LOGIC: Discrete (instant) step animation
            if not is_paused and move_index < len(moves):
                if current_time - last_move_time > speed_delay:
                    _, _, src, dst = moves[move_index]
                    poles[dst].append(poles[src].pop()) # TELEPORT
                    move_index += 1
                    last_move_time = current_time

        elif app_state == 'PLAYING_SMOOTH':
            # NEW EXTENSION LOGIC: Smooth, three-phase animation
            if not is_paused:
                
                # A: Initiate a new move
                if not animating and move_index < len(moves):
                    _, _, src, dst = moves[move_index]
                    disc_val = poles[src].pop() # Remove disc from the stack logically
                    
                    # Calculate start/target positions
                    start_x = POLE_X_POS[src]
                    start_y = BASE_Y - (len(poles[src]) + 1) * DISC_HEIGHT
                    
                    moving_disc_data = {
                        'val': disc_val, 'x': start_x, 'y': start_y, 
                        'target_pole': dst
                    }
                    animating = True
                    anim_phase = 0 # Start UP phase
                
                # B: Advance the current animation phase
                elif animating:
                    md = moving_disc_data
                    target_x = POLE_X_POS[md['target_pole']]
                    target_y_dest = BASE_Y - (len(poles[md['target_pole']]) + 1) * DISC_HEIGHT
                    
                    if anim_phase == 0: # UP
                        if md['y'] > CLEARANCE_Y: md['y'] -= move_speed
                        else: anim_phase = 1 # Finished UP
                            
                    elif anim_phase == 1: # ACROSS
                        if md['x'] < target_x: md['x'] = min(md['x'] + move_speed, target_x)
                        elif md['x'] > target_x: md['x'] = max(md['x'] - move_speed, target_x)
                        else: anim_phase = 2 # Finished ACROSS
                            
                    elif anim_phase == 2: # DOWN
                        if md['y'] < target_y_dest: md['y'] += move_speed
                        else:
                            poles[md['target_pole']].append(md['val']) # Place disc logically
                            animating = False
                            move_index += 1 # Advance to next move

        # --- Drawing ---
        if app_state == 'MENU':
            draw_menu()
        else:
            screen.fill(WHITE)
            # Draw UI, Poles (Drawing logic remains the same for both modes)
            
            # Draw Poles
            for x in POLE_X_POS:
                pygame.draw.line(screen, (80, 50, 20), (x, BASE_Y), (x, BASE_Y - 300), 10)
                pygame.draw.line(screen, (80, 50, 20), (x - 100, BASE_Y), (x + 100, BASE_Y), 6)

            # Draw Static Discs
            for i, stack in enumerate(poles):
                center_x = POLE_X_POS[i]
                for j, disc_val in enumerate(stack):
                    width = get_disc_width(disc_val)
                    rect_x = center_x - (width / 2)
                    rect_y = BASE_Y - (j + 1) * DISC_HEIGHT
                    
                    col = get_disc_color(disc_val, n_discs)
                    pygame.draw.rect(screen, col, (rect_x, rect_y, width, DISC_HEIGHT), border_radius=4)

            # Draw Moving Disc (only if animating)
            if animating and moving_disc_data:
                md = moving_disc_data
                width = get_disc_width(md['val'])
                draw_y = md['y'] - DISC_HEIGHT
                col = get_disc_color(md['val'], n_discs)
                pygame.draw.rect(screen, col, (md['x'] - width / 2, draw_y, width, DISC_HEIGHT), border_radius=4)
                pygame.draw.rect(screen, BLACK, (md['x'] - width / 2, draw_y, width, DISC_HEIGHT), 1, border_radius=4)


        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
if __name__ == "__main__":
    run_extended_hanoi()

## Conclusion

The main purpose of this project was to explore the abilities of recursive functions and animations, with the goal of using both to produce graphical, easily understood animations of the solutions to mathematical problems. The first three sections focused on developing these methods, using them to display images, while the last three focused on applying them to a problem. This goal has been achieved successfully, as we have been able to produce an interactive, developed animation of the solution to the Towers of Hanoi problem. Furthering understanding of the solutions to problems is the main purpose of animations, so adding interactive elements to the animations may have been the most useful part of achieving our aim; interactive animations allow a user to better understand how a solution works at their own pace. 

### Bibliography

$[1]$ Mandelbrot, B. (1978). "The fractal geometry of trees and other natural phenomena." *Geometrical Probability and Biological Structures: Buffon's 200th Anniversary: Proceedings of the Buffon Bicentenary Symposium on Geometrical Probability, Image Analysis, Mathematical Stereology, and Their Relevance to the Determination of Biological Structures,* 235–249.